In [1]:
import sys
import os

base_dir = os.path.abspath(os.path.join(os.path.dirname(''), '..'))
sys.path.append(base_dir)
os.environ['MU2E'] = base_dir

import os
from dotenv import load_dotenv

load_dotenv()

slack_api = os.getenv("SLACK_BOT_TOKEN") 

In [2]:
import mu2e;

/Users/annabelleboots/claude-env/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
loading the mu2e assistant


In [ ]:
import os
import time
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from datetime import datetime

# Initialize the Slack client
client = WebClient(token=slack_api)

# Define the channel ID for llm_tests
LLM_TESTS_CHANNEL_ID = "C07DXJHBVR9"  # Replace with actual channel ID

# Keep track of the latest timestamp we've processed
latest_ts = None

threads = {} # store thread_ts:last active

def fetch_new_messages():
    global latest_ts
    try:
        # Fetch history of the channel
        #ts_ = None if latest_ts == None else str(float(latest_ts)-(3600.*5))
        ts_ = latest_ts
        result = client.conversations_history(
            channel=LLM_TESTS_CHANNEL_ID,
            oldest= ts_
        )

        # Process new messages
        messages = result["messages"]
        #messages.reverse()  # To process older messages first

        for message in messages:
            if "bot_id" not in message and message['type'] == "message":  # Ignore bot messages
                if 'subtype' in message:
                    if message['subtype'] == "channel_join":
                        continue
                
                process_message(message)
                latest_ts = message["ts"]

        # check all active threads for new messages
        for ts in threads.keys():
            result = client.conversations_replies(
            channel=LLM_TESTS_CHANNEL_ID,
            ts=ts
            )
            messages = result["messages"]
            if "bot_id" not in messages[-1]:
                ts_latest = messages[-1]["ts"] # ts of latest message
                text = messages[-1]["text"]
                if float(ts_latest) > threads[ts]['ts']:
                    print("new message to answer to in thread")
                    process_with_llm(text, ts, threads[ts]['chat'])
            

    except SlackApiError as e:
        print(f"Error fetching messages: {e}")

def process_message(message):
    user = message["user"]
    text = message["text"]
    ts = message["ts"]
    typ = message["type"]
    if float(ts) < datetime.now().timestamp()-5.:
        return

    print(f"New message: {text}")
    #print(message)
    
    process_with_llm(text, ts)

def process_with_llm(text, thread_ts, chat=None):
    global threads
    print(f"Processing with LLM: {text}, ts: {thread_ts}")

    if chat is None:
         chat = mu2e.chat() # start a new chat if no chat exists yet

    
    answer = chat.ask(text)
    

    response_text = answer.content[0].text
    #response_text = "answer"

    result = client.chat_postMessage(
            channel=LLM_TESTS_CHANNEL_ID,
            thread_ts=thread_ts,
            text=response_text
        )
    if thread_ts not in threads:
         threads[thread_ts] = {'ts':datetime.now().timestamp(),
                               'chat':chat}
    else:
        threads[thread_ts]['ts'] = datetime.now().timestamp()

if __name__ == "__main__":
    while True:
        fetch_new_messages()
        time.sleep(5)  # Wait for 5 seconds before checking again

New message: hi
Processing with LLM: hi, ts: 1723748255.576349


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
